# Text and the City - Training

This notebook demonstrates how to fine-tune a BERT-like model for multi-label classification using an Excel file containing the "Text and the City" data.

Note: This notebook does not cover the extraction of content from PDF files. For that, please refer to the other provided notebook.

The notebook was executed on a free instance of Google Colab with 12.7 GB RAM, but it can be readily adapted to a regular Jupyter Notebook that can be run in any suitable environment.

## Install dependencies

In [ ]:
!pip install -q transformers[torch] datasets

## Load Data

To access your documents on Google Drive, you must grant Google Colab permission to your drive.

In the following code, we assume that you have a file named `10_Aufgabe_Innovationswettbewerb.xlsx` containing the required data. Feel free to change the file name as needed.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

DRIVE_DIR = "/content/drive/MyDrive/"

EXEL_FILE_NAME = "10_Aufgabe_Innovationswettbewerb.xlsx"

assert EXEL_FILE_NAME in os.listdir(DRIVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create a pandas DataFrame from the Excel file.

In [ ]:
import pandas as pd

xlsx_path = DRIVE_DIR + EXEL_FILE_NAME

df = pd.read_excel(xlsx_path, engine='openpyxl')
df

,Strategiefeld,Ziel,Stadtbezirk,Art des Textes,Titel,Inhalt,Name des pdf Dokuments,Kommentar
0,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Innenstadt als demokratisches Zentrum,Ludwigsvorstadt-Isarvorstadt,Antrag,Antrag: Schmellerwiese - Vereinfachtes schnell...,Begründung\nAm 25.07.2023 hat im Rahmen seiner...,23.11 A 1.1 Antrag Schmellerwiese_NACHTRAG_ges...,NaN
1,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Innenstadt,Freiraumquartierskonzept,Max-Joseph-Platz,"Für den Max-Joseph-Platz, bei dem es sich um e...",Konkretisierung der Konzeption „Freiraum M 203...,NaN
2,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Altstadt-Lehel,Antrag,Isartorplatz Umstellung der Mahd auf max. zwei...,Der BA1 Altstadt-Lehel beschließt:\nDer Bezirk...,22.08 A 6 Antrag Isartorplatz_Wiese.pdf,NaN
3,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Altstadt-Lehel,Antrag,Reparatur und Wiederinbetriebnahme des Fortuna...,Der Fortunabrunnen am Isartorplatz wird bis zu...,Antrag__Reparatur_und_Wiederinbetriebnahme_des...,NaN
4,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Altstadt-Lehel,Antrag,Hochwasserschutz im ersten Stadtbezirk prüfen,Die Landeshauptstadt München wird aufgefordert...,21.10 B 1.2 Antrag Hochwasserschutz.pdf,NaN
...,...,...,...,...,...,...,...,...
144,3 Innerstädtische Nutzungsmischung,Öffnung und erhöhte Nutzbarkeit von Gebäuden,Ludwigsvorstadt-Isarvorstadt,Antrag,Antrag: Denkmalschutz - Endlich die Sanierung ...,50 Jahre Bayerisches Denkmalschutzgesetz sollt...,23.02 C 1.1 Denkmalschutz - Endlich die Sanier...,NaN
145,3 Innerstädtische Nutzungsmischung,Öffnung und erhöhte Nutzbarkeit von Gebäuden,Ludwigsvorstadt-Isarvorstadt,Antrag,Schulhof-Öffnung Tumblingerschule – sofort for...,Begründung:\nErneut sind Anträge von Kindern b...,23.06 A 1.1 -Antrag-TumblingerSchulhof-Oeffnun...,NaN
146,3 Innerstädtische Nutzungsmischung,Öffnung und erhöhte Nutzbarkeit von Gebäuden,Ludwigsvorstadt-Isarvorstadt,Antrag,NaN,Der Bezirksausschuss möge beschließen\n1. Kühl...,23.07 D 1.1 Antrag_Kuehle_Raeume 2023 Nachtrag...,pdf hat keinen Titel
147,3 Innerstädtische Nutzungsmischung,Verträgliche Nachtkultur in der Innenstadt,Altstadt-Lehel,Beschlussentwurf,Entzerrung der Partyzone Tal in die Neuhauser ...,Die Bürgerversammlungsempfehlung zielt darauf ...,Beschlussentwurf Entzerrung Partyzone Tal.pdf,NaN


## Data Cleaning

Remove rows where the "Name des pdf Dokuments" (PDF names) column is not provided.

In [ ]:
df = df[df['Name des pdf Dokuments'].notnull()]

assert len(df[df['Inhalt'].isnull()].values) == 0
df

,Strategiefeld,Ziel,Stadtbezirk,Art des Textes,Titel,Inhalt,Name des pdf Dokuments,Kommentar
0,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Innenstadt als demokratisches Zentrum,Ludwigsvorstadt-Isarvorstadt,Antrag,Antrag: Schmellerwiese - Vereinfachtes schnell...,Begründung\nAm 25.07.2023 hat im Rahmen seiner...,23.11 A 1.1 Antrag Schmellerwiese_NACHTRAG_ges...,NaN
1,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Innenstadt,Freiraumquartierskonzept,Max-Joseph-Platz,"Für den Max-Joseph-Platz, bei dem es sich um e...",Konkretisierung der Konzeption „Freiraum M 203...,NaN
2,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Altstadt-Lehel,Antrag,Isartorplatz Umstellung der Mahd auf max. zwei...,Der BA1 Altstadt-Lehel beschließt:\nDer Bezirk...,22.08 A 6 Antrag Isartorplatz_Wiese.pdf,NaN
3,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Altstadt-Lehel,Antrag,Reparatur und Wiederinbetriebnahme des Fortuna...,Der Fortunabrunnen am Isartorplatz wird bis zu...,Antrag__Reparatur_und_Wiederinbetriebnahme_des...,NaN
4,"1 Übergeordnete Rahmenbedingungen, Klima und S...",Klimaanpassung,Altstadt-Lehel,Antrag,Hochwasserschutz im ersten Stadtbezirk prüfen,Die Landeshauptstadt München wird aufgefordert...,21.10 B 1.2 Antrag Hochwasserschutz.pdf,NaN
...,...,...,...,...,...,...,...,...
144,3 Innerstädtische Nutzungsmischung,Öffnung und erhöhte Nutzbarkeit von Gebäuden,Ludwigsvorstadt-Isarvorstadt,Antrag,Antrag: Denkmalschutz - Endlich die Sanierung ...,50 Jahre Bayerisches Denkmalschutzgesetz sollt...,23.02 C 1.1 Denkmalschutz - Endlich die Sanier...,NaN
145,3 Innerstädtische Nutzungsmischung,Öffnung und erhöhte Nutzbarkeit von Gebäuden,Ludwigsvorstadt-Isarvorstadt,Antrag,Schulhof-Öffnung Tumblingerschule – sofort for...,Begründung:\nErneut sind Anträge von Kindern b...,23.06 A 1.1 -Antrag-TumblingerSchulhof-Oeffnun...,NaN
146,3 Innerstädtische Nutzungsmischung,Öffnung und erhöhte Nutzbarkeit von Gebäuden,Ludwigsvorstadt-Isarvorstadt,Antrag,NaN,Der Bezirksausschuss möge beschließen\n1. Kühl...,23.07 D 1.1 Antrag_Kuehle_Raeume 2023 Nachtrag...,pdf hat keinen Titel
147,3 Innerstädtische Nutzungsmischung,Verträgliche Nachtkultur in der Innenstadt,Altstadt-Lehel,Beschlussentwurf,Entzerrung der Partyzone Tal in die Neuhauser ...,Die Bürgerversammlungsempfehlung zielt darauf ...,Beschlussentwurf Entzerrung Partyzone Tal.pdf,NaN


## Extract Labels

Retrieve all unique "Ziel" (goal) values from the dataframe and map each one to an index.

In [ ]:
labels = [label for label in df['Ziel'].unique()]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print("Number of labels: ", len(labels))
labels

Number of labels:  25


['Innenstadt als demokratisches Zentrum',
 'Klimaanpassung',
 'Klimaneutralität',
 'lebenswerte, gesunde, sichere, gerechte und lebendige Innenstadt für alle Bevölkerungs- und Nutzungsgruppen',
 'soziale Infrastrukturen für alle Bevölkerungs- und Nutzungsgruppen',
 'Erreichbarkeit einer sicheren und barrierefreien Innenstadt',
 'Freiraumentwicklung für Alle',
 'Identitätsort Innenstadt',
 'Innenentwicklung',
 'Mobilitätswende',
 'Multicodierte Freiräume',
 'Ökologische Aufwertung',
 'Erhaltung und Stärkung des Wirtschaftssystems durch resiliente, nachhaltige und vielfältige Strukturen',
 'Handel als tragende innerstädtische Funktion',
 'Hochattraktiver Einzelhandelsstandort Innenstadt',
 'Innenstadt für Kultur- und Kreativwirtschaft',
 'Innenstadt mit Flächen für Handwerk, Kleingewerbe und urbane Produktionsformen',
 'Innenstadt mit ökonomischer Vielfalt und innovativen Arbeitsformen',
 'Innenstadt mit vielfältiger Bildungslandschaft, Bildungsgerechtigkeit und Zugang zu Bildungsangebot

## Label Data

Retain only the "Name des pdf Dokuments" and "Inhalt" (content) columns from the dataframe and eliminate duplicate entries. Create a new column for each document named "labels", containing an array. The array has a length equal to the number of unique goals. Set the value to 1 when the index corresponds to the goal associated with that document and 0 otherwise.

In [ ]:
import numpy as np

no_duplicated_df = df.drop_duplicates(subset=['Name des pdf Dokuments'])[['Name des pdf Dokuments', "Inhalt"]]
#no_duplicated_df["label"] = [np.array([1 if len(df[('Name des pdf Dokuments' == name) & ("Ziel" == label)].values)else 0 for label in labels]) for name in no_duplicated_df['Name des pdf Dokuments'].values]

def create_labels_column(row):
  return [1.0 if len(df[(df['Name des pdf Dokuments'] == row['Name des pdf Dokuments']) & (df["Ziel"] == label)].values)else 0.0 for label in labels]

no_duplicated_df['labels'] = no_duplicated_df.apply(create_labels_column, axis=1)
no_duplicated_df

,Name des pdf Dokuments,Inhalt,labels
0,23.11 A 1.1 Antrag Schmellerwiese_NACHTRAG_ges...,Begründung\nAm 25.07.2023 hat im Rahmen seiner...,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Konkretisierung der Konzeption „Freiraum M 203...,"Für den Max-Joseph-Platz, bei dem es sich um e...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
2,22.08 A 6 Antrag Isartorplatz_Wiese.pdf,Der BA1 Altstadt-Lehel beschließt:\nDer Bezirk...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Antrag__Reparatur_und_Wiederinbetriebnahme_des...,Der Fortunabrunnen am Isartorplatz wird bis zu...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,21.10 B 1.2 Antrag Hochwasserschutz.pdf,Die Landeshauptstadt München wird aufgefordert...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
134,Beschluss_Innenstadt_19072006,Zur Stärkung des Wohnens in der Innenstadt wur...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
140,Beschlussentwurf Rischard Gelände.pdf,In beiden Anträgen wird die Aufstellung eines ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
141,22.07 C 3.2.3_Grundsatzbeschluss FSF.pdf,Der BA1 Altstadt-Lehel beschließt:\n1. Aufstel...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
147,Beschlussentwurf Entzerrung Partyzone Tal.pdf,Die Bürgerversammlungsempfehlung zielt darauf ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Rename the "Inhalt" column to "text" and retain only the "text" and "labels" columns.


In [ ]:
df = no_duplicated_df.rename(columns={'Inhalt': 'text'})[["text", "labels"]]
df = df.reset_index(drop=True)
df

,text,labels
0,Begründung\nAm 25.07.2023 hat im Rahmen seiner...,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"Für den Max-Joseph-Platz, bei dem es sich um e...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
2,Der BA1 Altstadt-Lehel beschließt:\nDer Bezirk...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Der Fortunabrunnen am Isartorplatz wird bis zu...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Die Landeshauptstadt München wird aufgefordert...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
72,Zur Stärkung des Wohnens in der Innenstadt wur...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
73,In beiden Anträgen wird die Aufstellung eines ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
74,Der BA1 Altstadt-Lehel beschließt:\n1. Aufstel...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
75,Die Bürgerversammlungsempfehlung zielt darauf ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## Split Dataset into Training and Validation

> NOTE:
>
> Since the least populated class in y has only 1 member, the dataset cannot be split in a stratified way. If more datapoints are available, try uncommenting `stratify=df["labels"].values`.

In [ ]:
import sklearn.model_selection

indices_train, indices_test = sklearn.model_selection.train_test_split(
    np.arange(df.shape[0]),
    random_state=42,
    shuffle=True,
    test_size=0.2,
    # stratify=df["labels"].values,
)

## Choose Model

Specify the name of a model available on Hugging Face.

Some options:
*   `microsoft/mdeberta-v3-base`
*   `bert-base-multilingual-cased`

Visit [this link](https://huggingface.co/models) for additional models. You can filter for language `German` and task `Classification` to explore other models.



In [ ]:
HUGGING_FACE_NAME = "bert-base-multilingual-cased"

## Load Tokenizer

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Convert the pandas dataframe into a tokenized dataset that can be directly input into the model.

> NOTE:
>
> The tokenizer truncates the vector representation of texts with more than 512 tokens, as this is the limit for BERT-like models. It might be worthwhile to explore whether the model performance improves by creating more data points using sliding/tumbling window techniques for texts exceeding 512 tokens. Conversely, if a text is represented in fewer than 512 tokens, padding is added at the end of the vector representation.

In [ ]:
import functools
import datasets

def tok_func(x, tokenizer):
    """
    tokenizes the field 'text' stored in x including padding
    """
    bert_max_token_n = 512
    return tokenizer(x["text"], padding="max_length", truncation=True, max_length=bert_max_token_n)

def get_dataset(ds, tok_func, tokenizer, indices_train, indices_test, train=True):
    """
    converts dataset to 'dataset' format required by Hugging Face

    Parameters:
    ----------
    ds: dataset
    tok_func: functiond use for tokenization
    indices_train: indices corresponding to the training set
    indices_test: indices corresponding to the training set
    train: if used for training

    Returns
    -------
    header of file
    """
    datasets_ds = datasets.Dataset.from_pandas(df)
    tok_function_partial = functools.partial(tok_func, tokenizer=tokenizer)
    tok_ds = datasets_ds.map(tok_function_partial, batched=True)
    if train:
        return datasets.DatasetDict(
            {
                "train": tok_ds.select(indices_train),
                "test": tok_ds.select(indices_test),
            }
        )
    else:
        return tok_ds

In [ ]:
# create Hugging Face 'dataset'
dataset = get_dataset(df, tok_func, tokenizer, indices_train, indices_test)

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

### Example of a Datapoint

In [ ]:
example = dataset['train'][0]
print(example.keys())

dict_keys(['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] Am 4. Januar 2021 hat das Bundesministerium für Verkehr und digitale Infrastruktur ( BMVI ) den vierten Förderaufruf zur Förderrichtlinie Städtische Logistik veröffentlicht. Das Ziel der Förderrichtlinie ist es, Landkreise und Kommunen dabei zu unterstützen, optimale Rahmenbedingungen für eine effiziente und nachhaltige städtische Logistik zu schaffen. Insbesondere sollen Luftschadstoffemissionen ( NOx ), Treibhausgasemissionen ( CO2 ), Feinstaubemissionen ( PM ) und Lärmemissionen, welche durch Lieferverkehre verursacht werden, reduziert werden und / oder der Verkehrsfluss verbessert werden. Die Erstellung eines Logistikkonzepts im Rahmen des BMVI - Programms zur Förderung der städtischen Logistik soll zum einen dazu beitragen, die Belastungen durch konventionelle Logistik in München und insbesondere in der Innenstadt zu senken. Zum anderen soll durch die Erstellung eines Konzepts der Weg für ein effizienteres System geschaffen werden. Denn neben einem leistungsstarken ÖPNV und

In [ ]:
example['labels']

[0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

## Load Model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(HUGGING_FACE_NAME,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Choose Hyperparameters

These parameters can be adjusted to achieve a more accurate model at the end of training.

In [ ]:
parameters = {}
parameters["learning_rate"] = 8e-5
parameters["batch_size"] = 8
parameters["weight_decay"] = 0.01
parameters["epochs"] = 1
# Could be considered
# parameters["warmup_ratio"] = 0.1
# parameters["cls_dropout"] = 0.3

FOLDER_TO_OUTPUT = "parametes"
parameters

{'learning_rate': 8e-05, 'batch_size': 8, 'weight_decay': 0.01, 'epochs': 1}

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    FOLDER_TO_OUTPUT,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=parameters["learning_rate"],
    per_device_train_batch_size=parameters["batch_size"],
    per_device_eval_batch_size=parameters["batch_size"],
    num_train_epochs=parameters["epochs"],
    weight_decay=parameters["weight_decay"],
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

## Define Metrics

This function defines the metrics used to evaluate the model and how these metrics are computed.

In this case, the f1 score, the ROC curve, and accuracy are computed. It is recommended to refer to the respective Wikipedia pages for more information on these metrics.

The overall metric used is the f1 score with micro average. For additional information and alternatives, refer to [this link](https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea).

The overall metric used can be changed; it has to be implemented in the `multi_label_metrics` function, and then the `metric_for_best_model` parameter in the cell above has to be set accordingly.

Keep in mind that the choice of the overall metric significantly influences how the model is trained.

> NOTE:
>
> The threshold value determines how high a value after the last sigmoid function must be to consider it a relevant goal for the given input text. This value can be adjusted as needed.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

## Fine-tune the Model

The model is trained on the training data and subsequently evaluated on the validation data.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.469477,0.205128,0.571041,0.125000


TrainOutput(global_step=8, training_loss=0.578583836555481, metrics={'train_runtime': 488.3081, 'train_samples_per_second': 0.125, 'train_steps_per_second': 0.016, 'total_flos': 16053088779264.0, 'train_loss': 0.578583836555481, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4694765508174896,
 'eval_f1': 0.20512820512820512,
 'eval_roc_auc': 0.5710414023757352,
 'eval_accuracy': 0.125,
 'eval_runtime': 27.0933,
 'eval_samples_per_second': 0.591,
 'eval_steps_per_second': 0.074,
 'epoch': 1.0}

## Check Overfitting

Evaluate on the training data.


In [ ]:
trainer.evaluate(eval_dataset=dataset["train"])

{'eval_loss': 0.4735867977142334,
 'eval_f1': 0.2742857142857143,
 'eval_roc_auc': 0.591581868640148,
 'eval_accuracy': 0.14754098360655737,
 'eval_runtime': 127.983,
 'eval_samples_per_second': 0.477,
 'eval_steps_per_second': 0.063,
 'epoch': 1.0}

In this scenario (actual numbers may vary), the training set's accuracy surpasses that of the validation dataset, indicating potential overfitting.

## Inference

This code can perform inference using the fine-tuned model.

In [ ]:
text = "Am 4. Januar 2021 hat das Bundesministerium für Verkehr und digitale Infrastruktur (BMVI) den vierten Förderaufruf zur Förderrichtlinie Städtische Logistik veröffentlicht. Das Ziel der Förderrichtlinie ist es, Landkreise und Kommunen dabei zu unterstützen, optimale Rahmenbedingungen für eine effiziente und nachhaltige städtische Logistik zu schaffen."

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

logits = outputs.logits
logits.shape

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Mobilitätswende']


## Final Remarks

*   The current number of labels is 25, resulting in 2<sup>25</sup> = 33,554,432 possible multi-label variations. Clustering the labels may be necessary.
*   The dataset is showing signs of overfitting, with low f1-score and accuracy. Consider augmenting the dataset size, even if further reduction in the number of labels is implemented.
*   Investigating the distribution of labels in the datapoints is crucial. Currently, it may not provide meaningful insights due to the imbalance between the numerous classes and limited datapoints. (histograms, confusion matrices)
*   As previously mentioned, certain aspects can be modified to enhance the model, including the choice of the base model, adjustment of hyperparameters, refining tokenization methods (summarizing before, windowing), fine-tuning thresholds, and optimizing output layer functions.